In [1]:
import time
import numpy as np
import pandas as pd
# from sklearn.cluster import DBSCAN
import pickle
import cv2
from deepface import DeepFace
import os
from retinaface import RetinaFace
from imutils import build_montages
from PIL import Image
from io import BytesIO
import gc

2024-07-03 16:18:50.211784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 16:18:50.521873: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 16:18:50.524708: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-03 16:18:50.909879: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-03 16:18:52.975837: W tensorflow/compiler/tf

In [2]:
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import storage

credentials = service_account.Credentials.from_service_account_file(
    "./sylvan-airship-406701-7556ece80fd0.json")
client = storage.Client(credentials=credentials)
bucket = client.get_bucket('full_images_2024')
blobs = bucket.list_blobs()

In [3]:
def process_images_from_directory(outlets, start_index=0, max_images=2000):
    for outlet in outlets:
        pre = outlet + '/' + outlet + '2018'
        count = 0
        blobs = bucket.list_blobs(prefix=pre)
        for blob in blobs:            
            if count < start_index:
                count+=1
                continue
            if count >= max_images:
                break
        
            print(f"{outlet}: {count}, name: {blob.name}")
            
            with BytesIO(blob.download_as_bytes()) as image_stream:
                with Image.open(image_stream) as image:
                    image = image.convert('RGB')  # Convert to RGB if not already
        
                    img_name = blob.name.split("/")[-1]
                    img_format = img_name.split(".")[-1]
        
                    np_image = np.array(image)  # Only convert to numpy array when necessary

                    face_objs = DeepFace.extract_faces(
                        img_path=np_image, 
                        detector_backend="yolov8",
                        enforce_detection = False,
                        
                    )
                    print(f"number of faces: {len(face_objs)}")
                    n = 1
                    for face in face_objs:
                        if face['confidence'] < 0.3:
                            continue
                        x, y, w, h = face['facial_area']['x'], face['facial_area']['y'], face['facial_area']['w'], face['facial_area']['h']
                        cropped_image = image.crop((x, y, x + w, y + h))
                        np_cropped_image = np.array(cropped_image)
                        cropped_img_name = f'{img_name.split(".")[0]}_alignment_{n}.{img_format}'
                        outlet_year = img_name.split("_")[0]
                        n += 1

                        for model in models:
                            embeddings[model][cropped_img_name] = []
                            embedding = DeepFace.represent(
                                img_path = np_cropped_image,
                                model_name = model,
                                detector_backend = 'yolov8', 
                                align = False,
                                enforce_detection = False,
                            )
                            for embed in embedding:
                                embeddings[model][cropped_img_name].append(embed['embedding'])
        
                        # Save to a BytesIO stream to avoid using local storage
                        cropped_stream = BytesIO()
                        cropped_image.save(cropped_stream, format=img_format)
                        cropped_stream.seek(0)
        
                        # Upload directly from BytesIO stream
                        new_blob = bucket.blob(f"alignment/{outlet}/{outlet_year}/{cropped_img_name}")
                        new_blob.upload_from_file(cropped_stream)
        
                        # Clear stream memory
                        cropped_stream.close()

        
                    if len(face_objs) == 0:
                        del np_image, face_objs, image, np_cropped_image, cropped_image
                    else:
                        del np_image, face_objs, image
                    gc.collect()
                    count+=1


In [ ]:
models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
]
embeddings = {model: {} for model in models}
# embeddings = pickle.loads(open("embeddings_alignment", "rb").read())
outlets = ["gatewaypundit", "huffpost", "nyt"]
process_images_from_directory(outlets, start_index = 0, max_images=200)

In [5]:
with open("embeddings_alignment_5", "wb") as f:
    f.write(pickle.dumps(embeddings))

In [6]:
len(embeddings['VGG-Face'])

2336